In [3]:
from sys import version, version_info
assert version_info >= (3, 7), "Need python >= 3.7, you have:\n{}".format(version_info)

# %load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
from time import sleep

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

In [4]:
@dataclass
class Pairs:
    japanese: str
    english: str
    

@dataclass
class Player:
    name: str
    vocab: List[Pairs] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))


In [11]:
_raw = """\
Healthy - けんこう
Calculate - けいさんする、計算する
Precise - せいかく、正確
Trick someone - だます、騙す
Free - むりょう, 無料
Trustable - しんよう (する), 信用【する】
A business - えいぎょう, 営業
As one would expect (/great) - さすが, 流石
International - こくさい, 国際
To entrust (to someone) - まかせる, 任せる
honest - しょうじき - 正直
feature - とくちょう - 特徴
basic - きほんてき - 基本的
to change - かえる - 変える
experiment - じっけん - 実験
efficiency - のうりつ - 能率
to improve - かいぜんする - 改善する
extreme - かげき - 過激
summary - おおすじ - 大筋
address - じゅうしょ - 住所
absence / non attendance - 欠席 / けっせき
rarely - めったに
to like / be pleased with - 気に入る / きにいる
grapes - ぶどう
sound / ring / roar - 鳴る / なる
board (board of wood) - 板 / いた
composition / essay - さくぶん　作文
surely certainly - きっと
to hold (hold the door) - 押さえる / おさえる
make a proposal / apply - 申し込む もうしこむ
beginner - しょきゅう - 初級
intermediate - ちゅうきゅう - 中級
advanced - じょうきゅう - 上級
to increase - ふえる - 増える
to decrease - へる - 減る
to select - えらぶ - 選ぶ
to continue - つづける - 続ける
to touch - さわる - 触る
rip-off - ぼったくり - ぼっ手繰り
finger/toe - ゆび - 指
Mostly, nearly - ほとんど - 殆ど
To be helpful - やくにたつ ー 役に立つ
To transfer (trains) / to change one's mind - のりかえる - 乗り換える
Humid, sultry - むしあつい - 蒸し暑い
To sweat - あせをかく- 汗をかく
To be bitten - さす - 刺す
Scissors - はさみ - 鋏
To become familiar with - なれる - 慣れる
Hard, solid, tough - かたい - 硬い
Short, brief(ly) - てみじか(に) - 手短(に)
be perplexed, get lost - まよう - 迷う
extremely/ considerably - ずいぶん - 随分
to let know / inform - しらせる - 知らせる
to come off / be removed - とれる - 取れる
remain over / be in excess - あまる - 余る
accumulate / get accumulated - つもる - 積もる
to lose (something abstract, like Dan's pride) - うしなう - 失う
shiny - こうたく - 光沢
burned / scorched / baked - やける - 焼ける
important - たいせつ - 大切
Bath cap - すいえいぼう - 水泳帽
Muscle - きんにく- 筋肉
Remote - えんかく - 遠隔
Knot - むすびめ - 結び目
Untie (a knot) - ほどく - ほどく
Remove (glasses), take off (a bra), undo,... (lots of meanings) - はずす - 外す
Swallow (bird) - つばめ - 燕
To be surprised - おどろく- 驚く
Wit, ressources, tact - きち- 機知
Usually, ordinarily - たいてい - 大抵
Tear, break open - やぶれる - 破れる
Not one cloud in the sky - かいせい - 快晴
Things to do, errand - ようじ - 用事
Present, gift - おくりもの - 贈り物
Cool, chill - ひやす - 冷やす
Put in order, dispose of - かたづける - 片付ける
Office - じむしつ - 事務室
Break off, fold, bend - おる - 折る
Thief, crook - どろぼう - 泥棒
For life, with all ones might - いっしょうけんめい - 一生懸命
to proceed - すすめる - 進める
to win - かつ - 勝つ
to lose (a competition) - まける - 負ける
to change (intransitive) - かわる - 変わる
schedule, plan - よてい - 予定
secret - ひみつ - 秘密
sugar - さとう - 砂糖
preparation - ようい - 用意
dark, gloomy - くらい - 暗い
for now - とりあえず - 取りあえず
hobby - しゅみ - 趣味
AI - じんこうちのう - 人工知能
chemistry - かがく - 化学
just barely, last minute - ぎりぎり - 限り限り
almost - ほぼ - 略
soon - まもなく - 間もなく
condition (prerequisite) - じょうけん - 条件
aim for - めざす - 目指す
be popular (esp. with the opposite sex) - もてる
no wonder, natural - あたりまえ - 当たり前
Toothpick - 爪楊枝 / つまようじ
Take off (clothes) - 脱ぐ / ぬぐ
One way (ticket) - 片道 / かたみち
Very / greatly - 大分 / だいぶ
Copy / photograph - 写す / うつす
Laundry - 洗濯物 / せんたくもの
Blanket - 毛布 / もうふ
Stoppage / stopping - 停車 / ていしゃ
I dont care (apathetic) - 別に気にしない / べつにきにしない
Crispy - カリカリ
Shrine - じんじゃ - 神社
Temple - てら - 寺
Spare time, free time - ひま - 暇
To fix, repair, service - しゅうり - 修理
Heal (but repair also) - なおす - 治す
To put in order, arrange - ととのえる - 整える
Prepare, get ready - じゅんび(する) - 準備(する)
Order (an item, a plate) - ちゅうもん - 注文
Real estate - ふどうさん - 不動産
Protest, objection - こうぎ - 抗議"""

pairs = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw.split("\n")]
]

In [12]:
%gui asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # TODO: learn how to use futures proprely
        try:
            future.result()
        except asyncio.InvalidStateError:
            future.set_result(0)
    widget.on_click(getvalue)
    return future

In [13]:
_players = ["Dan", "Romeo", "Paul"]
players = [Player(name=name, vocab=pairs) for name in _players]
shuffle(players)
# vocab_size = len(pairs) * len(players)
vocab_size = 10 * len(players)

flags = {"japanese": "🇯🇵", "english": "🇬🇧", "aussie": "🇦🇺"}

question = widgets.HTML(
    value="Questions here!",
)

list_players = widgets.Select(
    options=[p.name for p in players],
    value=players[0].name,
    disabled=True
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=vocab_size,
    step=1,
    description="[  0 /{:>3}]".format(vocab_size),
    bar_style='',
    orientation='horizontal'
)

status = widgets.ToggleButtons(
    options=['Known ', 'Unkown ', 'Skipped '],
    disabled=False,
    button_style='',
    icons=["check-circle", "question-circle", "forward"],
)
_mapping = {"Known ": "known", "Unkown ": "failed", "Skipped ": "skipped"}

next_b = widgets.Button(
    description='Next!',
    button_style='',
    icon='check'
)

async def f():
    for i in range(vocab_size):
        player_idx = i % len(players)
        curr_player = players[player_idx]
        list_players.value = curr_player.name
        word = curr_player.pick_word()

        q, a = choice([
            ["japanese", "english"],
            ["english", "japanese"],
        ])
        f_q, f_a = q, a
        if curr_player.name == "Dan":
            # Need the right flag
            f_q = "aussie" if q == "english" else f_q
            f_a = "aussie" if a == "english" else f_a

        question.value = "From <b>{}</b> {} to <b>{}</b> {}, translate:<p><b>{}</b>".format(
            q, flags[f_q], a, flags[f_a],
            getattr(word, q),
        )
        _ = await wait_for_change(next_b, 'value')
        next_b.disabled = True
        
        curr_player.update(_mapping[status.value])
        
        progress.value += 1
        progress.description = "[{:>3} /{:>3}]".format(progress.value, vocab_size)
        
        # Avoid quick double press
        sleep(0.5)
        next_b.disabled = False
        
    print("All done")
    
    for p in players:
        print("~"*80)
        print(f"Name: {p.name}")
        p.show_score()

asyncio.ensure_future(f())

display(progress)
display(question)
display(list_players)
display(status)
next_b

IntProgress(value=0, description='[  0 / 30]', max=30)

HTML(value='Questions here!')

Select(disabled=True, options=('Dan', 'Paul', 'Romeo'), value='Dan')

ToggleButtons(icons=('check-circle', 'question-circle', 'forward'), options=('Known ', 'Unkown ', 'Skipped '),…

Button(description='Next!', icon='check', style=ButtonStyle())

All done
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Dan
failed    :   1/120 [0.8%]
known     :   9/120 [7.5%]
skipped   :   0/120 [0.0%]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Paul
failed    :   0/120 [0.0%]
known     :  10/120 [8.3%]
skipped   :   0/120 [0.0%]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Name: Romeo
failed    :   1/120 [0.8%]
known     :   9/120 [7.5%]
skipped   :   0/120 [0.0%]


# Suggestions for improvements

- [ ] Timer: 20s
    + Put pressure
    - Put pressure
    - We're fast enough usually
    + only for Dan :D
- [ ] Type the words
    + ensure full knownledge
    ? hiragana only
    - make it slow
    ? maybe only for english -> japanese
- [ ] Issue with successive repeatitions
- [x] Keep choosing direction randomly
- [ ] Answer in a sentence, from japanese to english
- [x] Have Tatsuya ask the questions
- [x] Add minimal timer no to press next too often

